# Face Recognition 

In [2]:
!pip3 cache purge

Files removed: 0


In [3]:
!pip3 install torch torchvision --index-url https://download.pytorch.org/whl/cpu


Defaulting to user installation because normal site-packages is not writeable
Looking in indexes: https://download.pytorch.org/whl/cpu


In [4]:
!pip3 install tensorflow opencv-python-headless dlib face_recognition scikit-learn


Defaulting to user installation because normal site-packages is not writeable


In [5]:
!pip3 install facenet-pytorch


Defaulting to user installation because normal site-packages is not writeable


In [6]:
import cv2
import numpy as np
import torch
from facenet_pytorch import MTCNN, InceptionResnetV1
from sklearn.metrics.pairwise import cosine_similarity
import os


/Users/soubhikghosh/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(
/Users/soubhikghosh/Library/Python/3.9/lib/python/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [7]:
# Face detector
mtcnn = MTCNN(keep_all=True, device='cuda' if torch.cuda.is_available() else 'cpu')

# Face recognition model (FaceNet pre-trained)
facenet = InceptionResnetV1(pretrained='vggface2').eval()


In [8]:
def extract_face(image, mtcnn, target_size=(160, 160)):
    """
    Detects and crops the face from the image.
    """
    boxes, _ = mtcnn.detect(image)
    if boxes is None:
        return None
    cropped_faces = []
    for box in boxes:
        x1, y1, x2, y2 = [int(b) for b in box]
        face = image[y1:y2, x1:x2]
        face_resized = cv2.resize(face, target_size)
        cropped_faces.append(face_resized)
    return cropped_faces

def encode_faces(faces, facenet):
    """
    Generates embeddings for detected faces.
    """
    embeddings = []
    for face in faces:
        # Convert to PyTorch tensor
        face = torch.tensor(face.transpose(2, 0, 1)).float().unsqueeze(0) / 255.0
        embedding = facenet(face).detach().numpy()
        embeddings.append(embedding)
    return np.vstack(embeddings)


In [9]:
def recognize_faces(input_image, database, mtcnn, facenet, threshold=0.6):
    """
    Recognizes faces in the input image by comparing with the database.
    
    Parameters:
    - input_image: The image to process.
    - database: A dictionary of precomputed embeddings (name -> embedding).
    - mtcnn: The face detector.
    - facenet: The face recognition model.
    - threshold: Similarity threshold for recognition.
    
    Returns:
    - Annotated image with recognition results.
    """
    # Detect and crop faces
    faces = extract_face(input_image, mtcnn)
    if not faces:
        return input_image, []
    
    # Generate embeddings
    embeddings = encode_faces(faces, facenet)
    
    # Compare with database
    results = []
    for emb in embeddings:
        best_match = None
        best_score = 0
        for name, db_emb in database.items():
            score = cosine_similarity([emb], [db_emb])[0, 0]
            if score > best_score and score > threshold:
                best_match = name
                best_score = score
        results.append((best_match, best_score))
    
    # Annotate image
    annotated_image = input_image.copy()
    for (box, (name, score)) in zip(mtcnn.detect(input_image)[0], results):
        x1, y1, x2, y2 = [int(b) for b in box]
        label = f"{name} ({score:.2f})" if name else "Unknown"
        color = (0, 255, 0) if name else (0, 0, 255)
        cv2.rectangle(annotated_image, (x1, y1), (x2, y2), color, 2)
        cv2.putText(annotated_image, label, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)
    
    return annotated_image, results


In [10]:
def build_face_database(image_folder, mtcnn, facenet):
    """
    Creates a database of known faces from images in the given folder.
    """
    database = {}
    for file in os.listdir(image_folder):
        name, ext = os.path.splitext(file)
        if ext.lower() not in ['.jpg', '.png']:
            continue
        image = cv2.imread(os.path.join(image_folder, file))
        faces = extract_face(image, mtcnn)
        if faces:
            embeddings = encode_faces(faces, facenet)
            database[name] = embeddings[0]
    return database


In [ ]:
# Load known faces
database = build_face_database("known_faces", mtcnn, facenet)

# Process a test image
input_image = cv2.imread("chris_test.jpg")
annotated_image, results = recognize_faces(input_image, database, mtcnn, facenet)

# Display results
cv2.imshow("Face Recognition", annotated_image)
cv2.waitKey(0)
cv2.destroyAllWindows()

[ WARN:0@36.447] global loadsave.cpp:241 findDecoder imread_('chris_test.jpg'): can't open/read file: check file path/integrity


AttributeError: 'NoneType' object has no attribute 'size'

: 